## Analytical solution for the magnet configuration

In [6]:
#imports
import numpy as np
from scipy import special as scp
from math import factorial as factorial

In [7]:
class Magnets:
    """
    A class to simulate the cyllindrical magnets used in the Cd MOT.
    
    Attributes (all floats)
    ----------
    
    h: Height of the magnets in the z-direction (the beam direction) in cm
    R_out: outer radius of the magnets used in cm
    R_in: inner radius of the magnets used in cm
    M: magnetisation
    """
    def __init__(self, h=1., R_out=2., R_in=1.5, M=1):
        self.h = h
        self.R_out = R_out
        self.R_in = R_in
        self.M = M
        
    
    def ComputeField(self, r=10, z=10, n=10, gran=100): #n needs to be float for numpy to correctly use it for negative powers (floating point precision may not be great)
        """
        Computes the field for the given spatial range to a given accuracy.
        
        Attributes
        ----------
        r: range of coordinate r in cm (integer)
        z: range of coordinate z in cm (integer)
        n: number of terms in approximation (integer)
        gran: granularity - number of divisions of the r and z coordinates
        """
        if type(r) != int or type(z) != int:
            raise Exception("r and z can only be integers") # change in the future, maybe np arange
        r = np.linspace(0,r,gran) #create a list of r and z coordinates
        z = np.linspace(0,z,gran)
        n = np.arange(n) 
        self.n = n
        
        aMinus_Rlist = [] #for each r calculate the list of z-dependent a- coefficients 
        aPlus_Rlist = [] #and a+
        
        for Zcoord in z:
            aMinus_Rlist.append((-self.R_out**2)/(r**2+(self.h+Zcoord)**2))
            aPlus_Rlist.append((-self.R_out**2)/(r**2-(self.h+Zcoord)**2))
        
        potential_terms = [] #list of 2d lists (potential values on a component grid) for each term 
        for i in n:
            b = 1 + i
            x = 0.5 - (2*i)
            Rpotentials = [] #again a list of z-dependent values for each r
            for j in range(len(aMinus_Rlist)): #len(aMinus_Rlist) = gran ? minor speed improvement 
                #n-dependent term in the potential:
                scaling = (scp.factorial2(4*i-1)*((-1)**(-float(i)))*(r[j]**(2*i)))/((2**(2*i))*((factorial(i))**2))
                #calculate beta coefficients:
                betaMinus = ((r[j]**2+(self.h+z)**2)**(0.5-i)) * scp.betainc(aMinus_Rlist[j],b,x) #z and Rlist[j] should be done pairwise
                betaPlus = ((r[j]**2+(self.h-z)**2)**(0.5-i)) * scp.betainc(aPlus_Rlist[j],b,x)
                #calculate the z-dependent potential values list:
                potentials = 0.5*self.M * scaling * (betaPlus-betaMinus)
                Rpotentials.append(potentials) #append the potential values to potentials list
            print(len(Rpotentials))
            print(len(Rpotentials[0]))
            potential_terms.append(np.array(Rpotentials)) #append the potentials list to the master list
            print(len(potential_terms))
            #np.array structure neccessary for element-wise addition later on
        
        self.potential_terms = sum(potential_terms) #sum the elements of potential_terms element-wise to get one 2d list
        
        #self.field = #use sympy for differentiation
    
    def ChangeMag(self, newMag):
        self.M = newMag

In [8]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([[9,8,7],[6,5,4],[3,2,1]])
li = [a,b]
c = sum(li)
print(c)

[[10 10 10]
 [10 10 10]
 [10 10 10]]


In [9]:
a = np.linspace(0,10,100)
b = np.linspace(0,10,100)
c = 2

d = (c**2+a)*b
print(type(d)) #same length 

<class 'numpy.ndarray'>


In [10]:
magnet2 = Magnets()
magnet2.ComputeField()
magnet2.potential_terms

100
100
1
100
100
2
100
100
3
100
100
4
100
100
5
100
100
6
100
100
7
100
100
8
100
100
9
100
100
10


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [64]:
a = 5
print(float(a))

5.0
